https://adventofcode.com/2018/day/13

In [62]:
with open('data/13-1.txt') as fh:
    data = fh.read()

In [78]:
from collections import Counter

In [65]:
drxnlookup = {
    '>': 1,
    '^': 0+1j,
    '<': -1,
    'v': 0-1j
}

turnlookup = {
    (1, '/'): 0+1j,
    (0+1j, '/'): 1,
    (-1, '/'): 0-1j,    
    (0-1j, '/'): -1,

    (1, '\\'): 0-1j,
    (0-1j, '\\'): 1,
    (-1, '\\'): 0+1j,
    (0+1j, '\\'): -1
}

In [66]:
class Cart:
    def __init__(self, track, pos, drxnstr):
        self.track = track
        self.pos = self.original_pos = pos
        self.drxn = drxnlookup[drxnstr]
        
        self.intersections = 0
        self.dead = False
        
    def tick(self):
        self.pos = self.pos + self.drxn
        symbol = self.track.get(self.pos)
        if symbol == '+':
            self.drxn = self.drxn * [0+1j, 1, 0-1j][self.intersections % 3]
            self.intersections += 1
        elif symbol in ('/', '\\'):
            self.drxn = turnlookup[(self.drxn, symbol)]

    def __repr__(self):
        return '<Cart at %s headed %s>' % (self.pos, self.drxn)


In [67]:
def initialize(data=data):
    track = {}
    carts = []
    for (y, line) in enumerate(data.split('\n')):
        for (x, c) in enumerate(line.rstrip()):
            if c in ('+', '/', '\\'):
                track[complex(x, -y)] = c
            elif c in ('>', '^', '<', 'v'):
                cart = Cart(track, complex(x, -y), c)
                carts.append(cart)
    for cart in carts:
        cart.track = track
    return track, carts

In [68]:
def tick(carts):
    for cart in sorted(carts, key=lambda x: (-x.pos.imag, x.pos.real)):
        cart.tick()
        ctr = Counter(c.pos for c in carts)
        if len(ctr) != len(carts):
            raise Exception("Collision %s" % ctr.most_common())

In [69]:
testdata = (
r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   
""")

In [70]:
track, carts = initialize(testdata)

In [71]:
turn_symbol_lookup = {v: k for (k, v) in drxnlookup.items()}
cart_symbols = {c.pos: turn_symbol_lookup[c.drxn] for c in carts}

t1 = track.copy()
t1.update(cart_symbols)
for y in range(8):
    line = [t1.get(complex(x, -y), ' ') for x in range(16)]
    print(''.join(line))

for cart in carts:
    cart.tick()    

/ > \           
       /    \   
  / +  + \      
         v      
\ + /  \ +  /   
  \      /      
                
                


In [72]:
for _ in range(1_000_000):
    tick(carts)


Exception: Collision [((7-3j), 2)]

In [73]:
track, carts = initialize(data)

In [74]:
for _ in range(1_000_000):
    tick(carts)

Exception: Collision [((39-52j), 2), ((108-28j), 1), ((93-116j), 1), ((31-82j), 1), ((62-64j), 1), ((3-58j), 1), ((47-99j), 1), ((58-49j), 1), ((89-83j), 1), ((90-114j), 1), ((21-55j), 1), ((122-98j), 1), ((22-109j), 1), ((73-120j), 1), ((119-124j), 1), ((53-120j), 1)]

In [75]:
def tick2(carts):
    carts.sort(key=lambda x: (-x.pos.imag, x.pos.real))
    for cart in carts:
        if cart.dead:
            continue
        cart.tick()
        ctr = Counter(cart.pos for cart in carts if not cart.dead)
        for pos, ct in ctr.most_common():
            if ct == 1:
                break
            for cart in carts:
                if cart.pos == pos:
                    cart.dead = True
    return [cart for cart in carts if not cart.dead]
    

In [76]:
track, carts = initialize(data)

In [77]:
for _ in range(1_000_000):
    carts = tick2(carts)
    if len(carts) == 1:
        break
print(carts)        

[<Cart at (133-146j) headed (1+0j)>]
